In [1]:
import rasterio
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
#set working directory
os.chdir('E:\\landsat1')

In [4]:
#' At-Sensor Temperature or brightness temperature
#'
#' This function calculates at-Sensor Temperature or brightness temperature
#' @param Landsat_10 Raster* object, Landsat band 10
#' @param Landsat_11 Raster* object, Landsat band 11
#' @return A list containing brightness temperature corresponding to Landsat band 10 and Landsat band 11

#read landsat8 thermal bands
Band10=rasterio.open("band10.tif")
Band11=rasterio.open("band11.tif")
def BT(Band10,Band11):
    #convert to array for math operations
    band10= Band10.read(1).astype("float64")
    band11= Band11.read(1).astype("float64")

    l_lambda_10 = 3.3420*10**-4*band10 + 0.1
    l_lambda_11 = 3.3420*10**-4*band11 + 0.1

    BT_10 = (1321.0789/(np.log(774.8853/l_lambda_10 + 1)))
    BT_11 = (1201.1442/(np.log(480.8883/l_lambda_11 + 1)))
    return BT_10,BT_11
BT_10,BT_11 = BT(Band10,Band11)

In [5]:
#NDVI
#'
#'this function calculates NDVI 
#' @param Red Raster* object, red band of remote sensing imagery
#' @param NIR Raster* object, NIR band of remote sensing imagery

#read red and nir bands
red= rasterio.open("band4.tif")
nir= rasterio.open("band5_new.tif")

def NDVI(red, nir):
    #convert to array for math operations
    band4= red.read(1).astype("float64")
    band5= nir.read(1).astype("float64")
    #calculate ndvi
    ndvi= (band5-band4)/(band5+band4)
    return ndvi

ndvi=NDVI(red,nir) 

In [7]:
#' Proportion of vegetation or fractional vegetation cover
#'
#' Calculation of the proportion of vegetation or fractional vegetation cover from NDVI
#' @param NDVI Raster* object, NDVI calculated from remote sensing imagery
#' @param minNDVI = 0.2 (Ref. Sobrino et al. 2004)
#' @param maxNDVI = 0.5 (Ref. Sobrino et al. 2004)

def FV(ndvi):
    minNDVI = 0.2
    maxNDVI = 0.5
    FV= ((ndvi - minNDVI)/(maxNDVI - minNDVI))**2
    return FV
fv=FV(ndvi)

In [9]:
#' Land Surface Emissivity according to Van de Griend and Owe 1993
#'
#' This function calculates Land Surface Emissivity according to Van de Griend and Owe 1993
#' @param NDVI Raster* object, NDVI calculated from remote sensing imagery

def E_VandeGriend(ndvi):
    EV = 1.094 + 0.047*np.log(ndvi)
    #E_VandeGriend[E_VandeGriend>1] <- 1
    return EV
EV=E_VandeGriend(ndvi)

In [10]:
#' Atmospheric transmittance calculation
#'
#' This function calculates Atmospheric transmittance from near-surface air temperature (To, °C) and relative humidity (RH, %) of the date when Landsat passed over the study area
#' @param To Near-surface air temperature (°C) of the date when Landsat passed over the study area
#' @param RH relative humidity (%) of the date when Landsat passed over the study area
#' @param band A string specifying which Landsat 8 thermal band to use. It can be "band 10" or
#' "band 11"
#' @return Atmospheric transmittance
#' @examples
#' tau(To = 26, RH = 42, band = "band 11")
To= 26
To_K= To + 273.15
RH=42
def TAU(To,To_K,RH,band):
    RH_fraction= RH/100
    W= 0.0981*10*0.6108**((17.27*(To_K-273.15))/
                              (237.3+(To_K-273.15)))*RH_fraction+0.1697
    if band=="Band10":
        tau = -0.0164*W**2-0.04203*W+0.9715
    else:
        tau= -0.01218*W**2-0.07735*W+0.9603
    return tau
tau=TAU(To,To_K,RH,"Band10")

In [11]:
#' Mean atmospheric temperature
#'
#' This function calculates mean atmospheric temperature (Ta) using near-surface air
#' temperature (To)
#' @param To Near-surface air temperature (°C) of the date when Landsat passed over the study area
#' @param mod A string specifying which model to use. It can be anyone of "USA 1976 Standard" or
#' "Tropical Region" or "Mid-latitude Summer Region" or "Mid-latitude Winter Region"
#' @return Mean atmospheric temperature (K)
#' @examples
#' Ta(To = 26, mod = "Mid-latitude Winter Region")

def MAT (To, mod):
    To_K= To + 273.15
    if  mod == "USA 1976 Standard": 
        Ta = 25.940 + 0.8805*To_K
    elif mod == "Tropical Region":
        Ta = 17.977 + 0.9172*To_K
    elif mod == "Mid-latitude Summer Region":
        Ta = 16.011 + 0.9262*To_K
    else:
        Ta = 19.270 + 0.9112*To_K
    return Ta
Ta=MAT(To, "Mid-latitude Winter Region")

In [17]:
#Mono window algorithm
#'
#' This function calculates Land Surface Temperature(LST) using mono window algorithm
#' @param BT Raster* object, brightness temperature
#' @param tau Atmospheric transmittance
#' @param E Raster* object, Land Surface Emissivity calculated according to Van de Griend and Owe 1993 
#' @param Ta Mean atmospheric temperature (K) of the date when Landsat passed over the study area
#' @return LST
#' @examples
#' MWA(bt = Brightness Temperature)

def LST(bt, tau, EV, Ta):
    C= EV*tau
    D= (1-tau)*(1+(1-EV)*tau)
    LST_mwa = (-67.355351*(1-C-D)+(0.458606*(1-C-D)+C+D)*bt-D*Ta)/C
    return LST_mwa
LST_mwa=LST(BT_10, tau, EV, Ta)

In [16]:
LST_mwa_image = rasterio.open('LST_mwa_image.tiff','w',driver='Gtiff',
                          width=red.width, 
                          height = red.height, 
                          count=1, crs=red.crs, 
                          transform=red.transform, 
                          dtype='float64')
LST_mwa_image.write(LST_mwa,1)
LST_mwa_image2.close()